In [1]:
using Pkg; Pkg.activate("C:\\Users\\KarlH\\Desktop\\ComputationalPhysicsMaster\\OrbitalNeuralMethods")
using Revise
using OrbitalNeuralMethods
import OrbitalNeuralMethods as onm

  Activating project at `C:\Users\KarlH\Desktop\ComputationalPhysicsMaster\Code\OrbitalNeuralMethods`


In [2]:
import Random
rng = Random.MersenneTwister();

In [3]:
n = 4
ω = 0.25
ham = HOCoulomb(ω, shielding = 0.25);

l = 10
basis = SpinBasis(HOBasis(l, ω))
grid = [x for x in range(-10, stop = 10, length = 2001)]
system = System(n, basis, grid, ham);

mixer = Alpha(0.5)
rhf = RHF(system, mixer);
rhf = compute_ground_state!(rhf, max_iter = 100, tol = 1e-10, verbose=0);
energy(rhf)

4.466761475079295

In [4]:
slater = Slater(rhf);

In [7]:
function energy_opt(wf, optimizer)
    ham = HOCoulomb(0.25, shielding = 0.25);

    metro = Importance(equils = 1e4, samples = 1e6, step = 0.01);
    t = @elapsed wf_final, wf_opt, grad_results = optimize(wf, ham, metro, optimizer, nthreads = 12, verbose = true);
    println("\nGradient time: $(t) s")
    return wf_final, wf_opt
end

function energy_block(wf)
    samples_b = 2^23
    metro_b = Importance(equils = 1e6, samples = samples_b, step = 0.01);
    t = @elapsed result = blocking(wf, ham, metro_b, nthreads = 12);
    digits = Integer(-floor(log10(result.E_err)))
    print("$(round(result.E, digits=digits)) ± $(round(result.E_err, digits=digits)), $(round(result.E - 0.8247, digits=digits))")
    println("\nBlocking time: $(t) s")
end

energy_block (generic function with 1 method)

In [12]:
nn = NeuralNetwork([Dense(64), Tanh, Dense(1), Exp], n=n, rng = rng);
wf = SlaterNN(slater, nn);
optimizer = ADAM(wf, lr = 0.001, max_iter = 100, tol = 1e-6);
wf8_final, wf8_opt = energy_opt(wf, optimizer);

E = 4.428383 iter = 100/100                                      
Gradient time: 114.6110682 s


In [13]:
energy_block(wf8_final);
energy_block(wf8_opt);

5.3 ± 0.3, 4.5
Blocking time: 12.2106018 s
4.8 ± 0.2, 4.0
Blocking time: 12.2426715 s
